In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import numpy as np 
import pandas as pd
import matplotlib.dates as mdates
import seaborn as sns
from sklearn import preprocessing
import time
from datetime import datetime
from tqdm.auto import trange, tqdm
import os
import math
import datetime as dt
from numpy import newaxis
from keras.layers import Dense, Activation, Dropout, LSTM, GRU, SimpleRNN, BatchNormalization
from keras.models import Sequential, load_model, Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import matplotlib.pyplot as plt
from keras.optimizers import SGD,Adam
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras import backend as K
from keras.losses import MeanSquaredLogarithmicError
from keras import Input
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
!pip install pmdarima
from pmdarima.arima import ndiffs

In [0]:
# data path
rootPath = '/content/drive/My Drive/Colab Notebooks/NCML/'
trainFileName = 'input/train.csv'
testFileName = 'input/test.csv'
trainWithFlightFileName = 'train_old_population.csv'

In [0]:
def datelist(beginDate, endDate):
    # beginDate, endDate are something like ‘20160601’
    date_l=[datetime.strftime(x,'%Y-%m-%d') for x in list(pd.date_range(start=beginDate, end=endDate))]
    return date_l


def date_remove_year(dates):
    return [datetime.strftime(datetime.strptime(x, '%Y-%m-%d'),'%m-%d') for x in dates]


def dateStrs_to_dateframe(dates):
    return [datetime.strptime(x, '%Y-%m-%d') for x in dates]

In [0]:
def loadDataByCountry(Country, data_all=None, dateRange= ('2020-01-22','2020-04-13'), is_startFromFirstCaseDay=False, daily=False):
    confirmed_total_date = data_all[data_all['Country_Region']==Country].groupby(['Date']).agg({'ConfirmedCases':['sum']})
    flight_total_date = data_all[data_all['Country_Region']==Country].groupby(['Date']).agg({'DepartureFlight':['mean']})
    data = data_all[data_all['Country_Region']==Country].groupby(['Date']).agg({'ConfirmedCases':['sum'],
                                                                                'Fatalities':['sum'],
                                                                                # 'DepartureFlight':['mean'],
                                                                                # 'hospibed':['mean'],
                                                                                # 'lung':['mean'],
                                                                                # 'total_pop':['mean'],
                                                                                # 'density':['mean'],
                                                                                # 'age_100+':['mean']
                                                                                })
    day = flight_total_date.shape[0]
#     for i in tqdm(range(day), desc='Complete zero-value in flight data'):
#         if flight_total_date.iloc[i].values[0]==0:
#             flight_total_date.iloc[i] = int((flight_total_date.iloc[i-1]+flight_total_date.iloc[i+1])/2)
    
    is_FirstCase = False
    FirstCaseDate = dateRange[0]
    for i in tqdm(range(day), desc='Complete zero-value in data'):
        Date = data.iloc[i].name
        if not is_FirstCase:
            if data.iloc[i].values[1]!=0:
                print('For ', Country, 'The first case during the period just has been found', Date)
                FirstCaseDate = Date
                is_FirstCase = True
            
        # if data.iloc[i].values[2]==0:
        #     data.loc[Date,'DepartureFlight'] = int((data.iloc[i-1]['DepartureFlight']+data.iloc[i+1]['DepartureFlight'])/2)
    
    if daily:
        data_ConfirmedCases = data.ConfirmedCases.diff()
        data_Fatalities = data.Fatalities.diff()
        
        if not is_startFromFirstCaseDay:
            data.ConfirmedCases = data_ConfirmedCases
            data.Fatalities = data_Fatalities
            return data.iloc[1:].loc[dateRange[0]: dateRange[1]]
        else:
            data.ConfirmedCases = data_ConfirmedCases
            data.Fatalities = data_Fatalities
            return data.iloc[1:].loc[FirstCaseDate: dateRange[1]]
    
    if not is_startFromFirstCaseDay:   
        return data.loc[dateRange[0]: dateRange[1]]
    else:
        return data.loc[FirstCaseDate: dateRange[1]]

In [0]:
data_test = pd.read_csv(rootPath+testFileName)
data_all = pd.read_csv(rootPath+trainWithFlightFileName)
data_all.Province_State.fillna("None", inplace=True)
data_all = data_all.drop(['Province_State'], axis=1)
dayList =datelist ('20200122','20200413')

In [0]:
# ADF test
df_adftest = adfuller(train[:,0])
# df_adftest = adfuller(train[:,1])
df_adftest = pd.DataFrame({'Statistical test': [df_adftest[0]], 'P-Value': [df_adftest[1]], 'usedlag': [df_adftest[2]]})
df_adftest

In [0]:
# split data
split = 0.6
# 'China', 'US', 'Japan', 'United Kingdom', 'Korea. South', 'Italy'
country = 'China'
data = loadDataByCountry(Country = country, data_all=data_all, is_startFromFirstCaseDay=True, daily=False)
data = data.iloc[6:,:]
num_test = int((1-split)/2*(data.shape[0]))
num_train = data.shape[0]-num_test
indices = np.array(range(data.shape[0]))
test = data.values[indices[-num_test:]]
train = data.values[indices[:num_train]]

In [0]:
# plot ACF and get d, p
plt.rcParams.update({'figure.figsize':(12,5), 'figure.dpi':120})
# Original Series
fig, axes = plt.subplots(3, 2, sharex=True)
axes[0, 0].plot(train[:,0]); axes[0, 0].set_title('Original Series')
plot_acf(train[:,0], ax=axes[0, 1])
# 1st Differencing
axes[1, 0].plot(np.diff(train[:,0])); axes[1, 0].set_title('1st Order Differencing')
plot_acf(np.diff(train[:,0]), ax=axes[1, 1])
# 2nd Differencing
axes[2, 0].plot(np.diff(np.diff(train[:,0]))); axes[2, 0].set_title('2nd Order Differencing')
plot_acf(np.diff(np.diff(train[:,0])), ax=axes[2, 1])
plt.show()

# plt.rcParams.update({'figure.figsize':(12,5), 'figure.dpi':120})
# # Original Series
# fig, axes = plt.subplots(3, 2, sharex=True)
# axes[0, 0].plot(train[:,1]); axes[0, 0].set_title('Original Series')
# plot_acf(train[:,1], ax=axes[0, 1])
# # 1st Differencing
# axes[1, 0].plot(np.diff(train[:,1])); axes[1, 0].set_title('1st Order Differencing')
# plot_acf(np.diff(train[:,1]), ax=axes[1, 1])
# # 2nd Differencing
# axes[2, 0].plot(np.diff(np.diff(train[:,1]))); axes[2, 0].set_title('2nd Order Differencing')
# plot_acf(np.diff(np.diff(train[:,1])), ax=axes[2, 1])
# plt.show()

# Adf Test
print(ndiffs(train[:,0], test='adf'))
# KPSS test
print(ndiffs(train[:,0], test='kpss'))
# PP test:
print(ndiffs(train[:,0], test='pp'))

# # # Adf Test
# print(ndiffs(train[:,1], test='adf'))
# # KPSS test
# print(ndiffs(train[:,1], test='kpss'))
# # PP test:
# print(ndiffs(train[:,1], test='pp'))

In [0]:
# plot PACF and get q
plt.rcParams.update({'figure.figsize':(12,2), 'figure.dpi':120})
fig, axes = plt.subplots(1, 2, sharex=True)
axes[0].plot(np.diff(np.diff(train[:,0]))); axes[0].set_title('2nd Differencing')
axes[1].set(ylim=(0,5))
plot_pacf(np.diff(np.diff(train[:,0])), ax=axes[1])
plt.show()

# plt.rcParams.update({'figure.figsize':(12,2), 'figure.dpi':120})
# fig, axes = plt.subplots(1, 2, sharex=True)
# axes[0].plot(np.diff(np.diff(train[:,1]))); axes[0].set_title('2nd Differencing')
# axes[1].set(ylim=(0,5))
# plot_pacf(np.diff(np.diff(train[:,1])), ax=axes[1])
# plt.show()

In [0]:
from statsmodels.tsa.arima_model import ARIMA
# Build Model
model = ARIMA(train[:,0], order=(1, 2, 1)) # p d q
# model = ARIMA(train[:,1], order=(1, 2, 1)) # p d q
fitted = model.fit(disp=-1)  
# Forecast
fc, se, conf = fitted.forecast(num_test, alpha=0.05)  # 95% conf

indice = date_remove_year(data.index)
# Make as pandas series
fc_series = pd.Series(fc, index=indice[-num_test:])
lower_series = pd.Series(conf[:,0], index=indice[-num_test:])
upper_series = pd.Series(conf[:,1], index=indice[-num_test:])
train_p = pd.Series(train[:,0], index=indice[:num_train])
test_p = pd.Series(test[:,0], index=indice[-num_test:])
# train_p = pd.Series(train[:,1], index=indice[:num_train])
# test_p = pd.Series(test[:,1], index=indice[-num_test:])

# Plot
fig, ax = plt.subplots(figsize=(12,5))
# ax.figure(figsize=(6,5), dpi=100)
ax.plot(train_p, label='training')
ax.plot(test_p, label='test')
ax.plot(fc_series, label='forecast')
ax.fill_between(lower_series.index, lower_series, upper_series, color='k', alpha=.15)
ax.set_xticklabels(indice, rotation=45, size=5)
ax.set_title(country+' ARIMA New Cases Forecast Result') # Daily or not
# ax.set_title(country+' ARIMA Fatalities Forecast Result')
ax.legend(loc='upper left', fontsize=8)
ax.set_ylabel("Number of cases", size=13)
ax.set_xlabel("Date", size=13)
fig.show()

In [0]:
from sklearn.metrics import mean_squared_log_error
np.around(mean_squared_log_error(test[:,0],fc), decimals=10)
# np.around(mean_squared_log_error(test[:,1],fc), decimals=10)

In [0]:
print(fitted.summary())